# Coding Lecture 3

Today's goal:
- (Review) Iterations to prepare the implementation of a simple gradient descent algorithm.
- `PyTorch` autograd.

In [ ]:
import torch
import math
import time
from tqdm.auto import tqdm

torch.cuda.manual_seed(42)

## Essential components of gradient descent

Example: linear regression. Use a cubic polynomial to fit a $f(x) :=\sin(x)$ function on $D = (-\pi, \pi)$. That is:

$$\min_{p\in \mathbb{P}^3} \|p - f\|^2_{L^2(D)} $$

Equivalently, this is approximated as

$$\min_{(a,b,c,d)\in \mathbb{R}^4 } \sum_{x\in D_h} \Delta x\, |a+bx+cx^2+dx^3 - f(x)|^2 $$

In [ ]:
dtype = torch.float # single-precision float number
device = torch.device("cpu")

# Create input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
delta_x = x[1] - x[0]
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-3
N_iter = 2000
with tqdm(total=N_iter) as pbar:
    for t in range(N_iter):
        # Forward pass: compute predicted y
        y_pred = a + b * x + c * x ** 2 + d * x ** 3

        # Compute and print loss
        loss = (delta_x*(y_pred - y).pow(2).sum())

        # Backprop (chain rule) to compute gradients of a, b, c, d with respect to loss
        grad_y_pred = 2.0 * delta_x * (y_pred - y)
        grad_a = grad_y_pred.sum()
        grad_b = (grad_y_pred * x).sum()
        grad_c = (grad_y_pred * x ** 2).sum()
        grad_d = (grad_y_pred * x ** 3).sum()

        # Update weights using gradient descent
        a -= learning_rate * grad_a
        b -= learning_rate * grad_b
        c -= learning_rate * grad_c
        d -= learning_rate * grad_d
        pbar.set_description(f"loss: {loss.item():.6f}")
        pbar.update()
        time.sleep(2e-3)


print(f'Result: y approx = {a.item():.3f} + {b.item():.3f} x + {c.item():.3f} x^2 + {d.item():.3f} x^3')

## How to compute gradient in PyTorch: autograd

Let's take a look at how ``autograd`` collects gradients. We create two tensors ``a`` and ``b`` with
``requires_grad=True``. This signals to ``autograd`` that every operation on them should be "tracked".

Reference: 
- adapted from the official tutorial

In [ ]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

We create another tensor ``f`` from ``a`` and ``b`` elementwisely.

\begin{align}f = 3a^3 - b^2\end{align}

or let $f = (f_i)^{\top}$ and for each $i$

\begin{align}f_i = 3a_i^3 - b_i^2\end{align}

In [ ]:
f = 3*a**3 - b**2

print(f)

``a`` and ``b`` are usually parameters of a neural network, and ``f``
to be the ``loss function`` (similar to the error/residual above). In gradient based methods, we want gradients of each component of $f$ w.r.t. parameters, i.e.

\begin{align}\frac{\partial f_i}{\partial a_i} = 9a_i^2\end{align}

\begin{align}\frac{\partial f_i}{\partial b_i} = -2b_i\end{align}


When we call ``.backward()`` on ``f``, autograd calculates these gradients
and stores them in the respective tensors' ``.grad`` attribute.

We need to explicitly pass a ``gradient`` argument in ``f.backward()`` because it is a vector.
``gradient`` is a tensor of the same shape as ``f``, and it represents the
gradient of $f$ w.r.t. itself, i.e.

\begin{align}\frac{df_i}{df_i} = 1\end{align}

Equivalently, we can also aggregate $f$ into a scalar and call backward implicitly, like ``f.sum().backward()``.

In [ ]:
external_grad = torch.tensor([1., 1.]) # first component and second component modifier
f.backward(gradient=external_grad)

Gradients are now deposited in ``a.grad`` and ``b.grad``


In [ ]:
# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)
print(a.grad)
print(b.grad)

In [ ]:
a.grad = None
b.grad = None

Now set $g = \sum f_i$

In [ ]:
g = (3*a**3 - b**2).sum()

In [ ]:
g.backward()

In [ ]:
print(a.grad)
print(b.grad)

## Implementation of GD using autograd



In [ ]:
# Create input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
delta_x = x[1] - x[0]
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-3
N_iter = 2000
with tqdm(total=N_iter) as pbar:
    for t in range(N_iter):
        # Forward pass: compute predicted y
        y_pred = a + b * x + c * x ** 2 + d * x ** 3
        # for p in [a, b, c, d]:
        #     p.grad = None

        # Compute and print loss
        loss = (delta_x*(y_pred - y).pow(2).sum())

        # Backprop (chain rule) through autograd
        loss.backward()

        # Update weights using gradient descent
        with torch.no_grad(): # this is necessary if autograd attribute is called manually
            a -= learning_rate * a.grad
            b -= learning_rate * b.grad
            c -= learning_rate * c.grad
            d -= learning_rate * d.grad
        
            # manually zeroing the grad stored in each variable
            a.grad.zero_()
            b.grad.zero_()
            c.grad.zero_()
            d.grad.zero_()
        
        pbar.set_description(f"loss: {loss.item():.6f}")
        pbar.update()
        time.sleep(2e-3)


print(f'Result: y approx = {a.item():.3f} + {b.item():.3f} x + {c.item():.3f} x^2 + {d.item():.3f} x^3')

## Implementation using matmul

Data matrix $X$:

$$
X_{i, \cdot}  = [1, x_i, x_i^2, x_i^3 ], y_i = f(x_i), \text{ and } w = [a, b, c, d]^{\top} 
$$
then the regression becomes 
$$
\min_{w\in \mathbb{R}^4}\Delta x\, \|Xw - y\|^2
$$


In [1]:
# code here